In [4]:
import pandas as pd
import pickle
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer


In [5]:
with open('finalized_model.sav', 'rb') as fid:
    model = pickle.load(fid)

EOFError: Ran out of input

In [6]:
courses_df = pd.read_csv("clean_phone_data.csv")
dt = {
    # string
    'colors': 'string',
    'build_material': 'string',
    'has_wifi': 'string',
    'has_bluetooth': 'string',
    'has_cardslot': 'string',
    'camera_list_resolution_back': 'string',
    'camera_list_resolution_front': 'string',
    'camera_resolution_back_recording': 'string',
    'camera_resolution_front_recording': 'string',
    'os': 'string',
    'storage_type': 'string',
    'brand': 'string',

    # float
    'battery_size': 'float',
    'battery_endurance': 'float',
    'dimensions_height': 'float',
    'dimensions_width': 'float',
    'dimensions_depth': 'float',
    'weight': 'float',
    'screen_size': 'float',
    'screen_resolution': 'float',
    'price_min': 'float',
    'price_max': 'float',
    'lazada_ratings': 'float',

    # int
    'camera_count_back': 'int',
    'camera_count_front': 'int',
    'benchmark_antutu': 'int',
    'benchmark_geekbench': 'int',
    'ram_min': 'int',
    'ram_max': 'int',
    'storage_min': 'int',
    'storage_max': 'int',
    'lazada_reviews': 'int',
    'release': 'int'

}

courses_df = courses_df.astype(dt)

comb_frame = courses_df.select_dtypes('string').agg(' '.join, axis=1)
comb_frame = comb_frame.replace({"[^A-Za-z0-9 ]+": ""}, regex=True)

# Add clustering labels to every non-retired course
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(comb_frame)

course_df = pd.read_csv("clean_phone_data.csv")
course_df = course_df.astype(dt)

course_df['InputString'] = course_df.select_dtypes('string').agg(' '.join, axis=1)


course_df['ClusterPrediction'] = ""


def cluster_predict(str_input):
    Y = vectorizer.transform(list(str_input))
    prediction = model.predict(Y)
    return prediction

# Cluster category for each live course
course_df['ClusterPrediction'] =  course_df.apply(lambda x: cluster_predict(course_df['InputString']), axis=0)


def recommend_util(str_input):
    
    # Predict category of input string category
    temp_df = course_df.copy()
    temp_df = temp_df.astype(dt)
    temp_df['InputString'] = temp_df.select_dtypes('string').agg(' '.join, axis=1)
    str_input = list(temp_df['InputString'])
    
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    
    temp_df = course_df.loc[course_df['ClusterPrediction'] == prediction_inp]
    temp_df = temp_df.sample(10)
    
    return list(temp_df['CourseId'])

if __name__ == '__main__':
    queries = ['wp7-core', 'ef41-data-access', 'nosql-big-pic', 'procedural-ice-modeling-softimage-153', \
               'beginners-guide-shading-networks-softimage-510', 'centralized-logging-elastic-stack', \
               'apache-pig-data-transformations']

    for query in queries:
        res = recommend_util(query)
        print(res)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'dimensions_height' not found in columns."

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import pickle
from sklearn.cluster import KMeans


data = pd.read_csv('clean_phone_data.csv')

# set data types
dt = {
    # string
    'details': 'string',
    'colors': 'string',
    'build_material': 'string',
    'has_wifi': 'string',
    'has_bluetooth': 'string',
    'has_cardslot': 'string',
    'camera_list_resolution_back': 'string',
    'camera_list_resolution_front': 'string',
    'camera_resolution_back_recording': 'string',
    'camera_resolution_front_recording': 'string',
    'os': 'string',
    'storage_type': 'string',
    'brand': 'string',

    # float
    'battery_size': 'float',
    'battery_endurance': 'float',
    'dimensions_height': 'float',
    'dimensions_width': 'float',
    'dimensions_depth': 'float',
    'weight': 'float',
    'screen_size': 'float',
    'screen_resolution': 'float',
    'price_min': 'float',
    'price_max': 'float',
    'lazada_ratings': 'float',

    # int
    'camera_count_back': 'int',
    'camera_count_front': 'int',
    'benchmark_antutu': 'int',
    'benchmark_geekbench': 'int',
    'ram_min': 'int',
    'ram_max': 'int',
    'storage_min': 'int',
    'storage_max': 'int',
    'lazada_reviews': 'int',
    'release': 'int'

}




details_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['details'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

colors_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['colors'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

build_material_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['build_material'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

has_wifi_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['has_wifi'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

has_bluetooth_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['has_bluetooth'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

has_cardslot_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['has_cardslot'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_list_resolution_back_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_list_resolution_back'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_list_resolution_front_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_list_resolution_front'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_back_recording_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_resolution_back_recording'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_front_recording_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_resolution_front_recording'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

os_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['os'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

storage_type_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['storage_type'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

brand_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['brand'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

feature_union = FeatureUnion([
    ('details', details_pipeline),
    ('colors', colors_pipeline),
    ('build_material', build_material_pipeline),
    ('has_wifi', has_wifi_pipeline),
    ('has_bluetooth', has_bluetooth_pipeline),
    ('has_cardslot', has_cardslot_pipeline),
    ('camera_list_resolution_back', camera_list_resolution_back_pipeline),
    ('camera_list_resolution_front', camera_list_resolution_front_pipeline),
    ('camera_resolution_back_recording', camera_resolution_back_recording_pipeline),
    ('camera_resolution_front_recording', camera_resolution_front_recording_pipeline),
    ('os', os_pipeline),
    ('storage_type', storage_type_pipeline),
    ('brand', brand_pipeline)
])

# Create a pipeline that extracts features from the data then creates a model
pipeline = Pipeline([
    ('union', feature_union),
    ('model', KMeans(n_clusters=150, random_state=42, n_init=100))
])

# Fit the pipeline to the data
pipeline.fit(data)



Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('details',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function <lambda> at 0x000002CEAB16AC00>)),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(stop_words='english'))])),
                                                ('colors',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function <lambda> at 0x000002CEAB16AD40>)),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(stop_word...
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function <lambda> at 0x000002CEAB181120>)),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(stop_words='english'))])),
                                                ('brand',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function <lambda> at 0x000002CEAB1811C0>)),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(stop_words='english'))]))])),
                ('model', KMeans(n_clusters=150, n_init=100, random_state=42))])

In [10]:

# test the model
target = {
    'details': 'red',
    'colors': 'red',
    'build_material': '',
    'has_wifi': '',
    'has_bluetooth': '',
    'has_cardslot': '',
    'camera_list_resolution_back': '',
    'camera_list_resolution_front': '',
    'camera_resolution_back_recording': '',
    'camera_resolution_front_recording': '',
    'os': 'android',
    'storage_type': '',
    'brand': '',

}

# transform the new phone
target_df = pd.DataFrame([target])

# predict the cluster
prediction = pipeline.predict(target_df)


# get the full details of the cluster
cluster_data = data[pipeline.named_steps['model'].labels_ == prediction[0]]


cluster_data[:10]


,phone,brand,colors,model,battery_size,battery_endurance,weight,os,release,screen_size,...,storage_type,price_min,price_max,lazada_ratings,lazada_reviews,build_material,has_wifi,has_bluetooth,has_cardslot,details
368,realme_c30-11621,Realme,Lake Blue|Bamboo Green,C30,5000,103.074491,182.0,Android,2022,6.50,...,UFS,14046.257103,25564.426458,4.630137,73,Glass|plastic|plastic,b|g|n,5.0,Yes,//www.lazada.com.ph/products/realme-c30-phone2...
369,realme_c30s-11878,Realme,Stripe Blue|Stripe Black,C30s,5000,103.074491,186.0,Android,2022,6.50,...,UFS,5699.000000,6025.000000,5.000000,27,Glass|plastic|plastic,b|g|n,4.2,Yes,//www.lazada.com.ph/products/realme-c30s-32gb-...
391,realme_narzo_50i-11123,Realme,Carbon Black|Mint Green,Narzo 50i,5000,103.074491,195.0,Android,2021,6.50,...,UFS,5422.500000,6490.000000,0.000000,0,Glass|plastic|plastic,b|g|n,4.2,Yes,//www.lazada.com.ph/products/realme-narzo-50i-...
392,realme_narzo_50i_prime-11630,Realme,Dark Blue|Mint Green,Narzo 50i Prime,5000,103.074491,182.0,Android,2022,6.50,...,UFS,14046.257103,25564.426458,0.000000,0,Glass|plastic|plastic,b|g|n,5.0,Yes,//www.lazada.com.ph/products/cod-realme-narzo-...
540,ulefone_note_15-12378,Ulefone,Sunset Purple|Diamond Blue|Twilight Black,Note 15,4000,82.459593,179.0,Android,2023,6.22,...,UFS,4215.000000,6022.000000,0.000000,0,Glass|plastic|plastic,b|g|n,4.0,Yes,//www.lazada.com.ph/products/ulefone-note-15-2...


In [ ]:

# Define the specifications of the target phone
target_specs = {
    'details': 'Some details',
    'colors': 'Red',
    
}

# Create a DataFrame for the target phone specs
target_df = pd.DataFrame([target_specs])

# Transform the target phone specs using the previously defined FeatureUnion
target_feature_vectors = feature_union.transform(target_df)


# Calculate cosine similarity between the target phone specs and all items in the dataset
similarities_target = cosine_similarity(target_feature_vectors, feature_vectors)

# Sort indices of items by similarity
similar_indices_target = similarities_target.argsort()[0][::-1]

# Print top recommendations
print("Top Recommendations based on Target Phone Specs:")
for index in similar_indices_target[:10]:
    print(f"Brand: {data.iloc[index]['brand']}, Model: {data.iloc[index]['model']}")



Top Recommendation based on Non-Existing Phone Specs:
Brand: vivo, Model: Y78+
Brand: OnePlus, Model: Nord 2T
Brand: Oppo, Model: A18
Brand: Oppo, Model: A17k
Brand: Oppo, Model: A17
Brand: Oppo, Model: A16K
Brand: Oppo, Model: A16
Brand: Oppo, Model: A15s
Brand: OnePlus, Model: Open
Brand: OnePlus, Model: Nord CE 3 Lite


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

# Sample data

data = pd.read_csv('clean_phone_data.csv')

df = pd.DataFrame(data)

# add a column for the average price
df['price_avg'] = (df['price_min'] + df['price_max']*2) / 3

# add a column for the average ram  
df['ram_avg'] = (df['ram_min'] + df['ram_max']*2) / 3

# add a column for the average storage
df['storage_avg'] = (df['storage_min'] + df['storage_max']*2) / 3

# make all columns string
df = df.astype(str)

# Define separate vectorizers for brand and model
brand_vectorizer = TfidfVectorizer()
model_vectorizer = TfidfVectorizer()

# Create a pipeline for each feature

model_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['model'], validate=False)),
    ('tfidf', model_vectorizer)
])


brand_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['brand'], validate=False)),
    ('tfidf', brand_vectorizer)
])

operating_system_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['os'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

release_year_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['release'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

color_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['colors'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

ram_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['ram_avg'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

rom_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['storage_avg'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

screen_type_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['screen_type'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

screen_size_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['screen_size'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

battery_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['battery_size'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

battery_endurance_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['battery_endurance'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

weight_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['weight'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_count_back_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_count_back'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_count_front_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_count_front'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_back_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_list_resolution_back'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_front_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_list_resolution_front'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_back_recording_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_resolution_back_recording'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

camera_resolution_front_recording_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['camera_resolution_front_recording'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

price_pipeline = Pipeline([
    ('selector', FunctionTransformer(lambda x: x['price_avg'], validate=False)),
    ('tfidf', TfidfVectorizer(stop_words='english'))
])








# Combine the pipelines using FeatureUnion
feature_union = FeatureUnion([
    ('brand', brand_pipeline),
    ('model', model_pipeline),
    ('os', operating_system_pipeline),
    ('release', release_year_pipeline),
    ('colors', color_pipeline),
    ('ram', ram_pipeline),
    ('rom', rom_pipeline),
    ('screen_type', screen_type_pipeline),
    ('screen_size', screen_size_pipeline),
    ('battery_size', battery_pipeline),
    ('battery_endurance', battery_endurance_pipeline),
    ('weight', weight_pipeline),
    ('price', price_pipeline)
])


# Fit and transform the data
feature_vectors = feature_union.fit_transform(df)

# Now feature_vectors contains TF-IDF representations for both 'brand' and 'model'
from sklearn.metrics.pairwise import cosine_similarity


# Define the non-existing phone specs
non_existing_phone_specs = {
    'details': '',
    'brand': '',
    'model': '', 
    'os': '',
    'release': '',
    'colors': 'red',
    'ram_avg': '',
    'storage_avg': '',
    'screen_type': '',
    'screen_size': '',
    'battery_size': '',
    'battery_endurance': '',
    'weight': '',
    'price_avg': '10000' 

}

# Create a DataFrame for the non-existing phone specs
non_existing_df = pd.DataFrame([non_existing_phone_specs])

# Transform the non-existing phone specs using the previously defined FeatureUnion
non_existing_feature_vectors = feature_union.transform(non_existing_df)

# Calculate cosine similarity between the non-existing phone specs and all items in the dataset
similarities_non_existing = cosine_similarity(non_existing_feature_vectors, feature_vectors)

# Sort indices of items by similarity
similar_indices_non_existing = similarities_non_existing.argsort()[0][::-1]

# Print top recommendation
print("Top Recommendation based on Non-Existing Phone Specs:")
df[['model', 'brand', 'release', 'price_avg']].iloc[similar_indices_non_existing][:10]


Top Recommendation based on Non-Existing Phone Specs:


,model,brand,release,price_avg
822,Y78+,vivo,2023,16457.5
269,Nord 2T,OnePlus,2022,20893.0
279,A18,Oppo,2023,21725.03667293065
278,A17k,Oppo,2022,6988.0
277,A17,Oppo,2022,13886.039333333332
276,A16K,Oppo,2021,21686.399999999998
275,A16,Oppo,2021,13673.406
274,A15s,Oppo,2020,6654.666666666667
273,Open,OnePlus,2023,6685.3
272,Nord CE 3 Lite,OnePlus,2023,14593.0


In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Sample data

data = pd.read_csv('clean_phone_data.csv')

df = pd.DataFrame(data)

# add a column for the average price
df['price_avg'] = (df['price_min'] + df['price_max']*2) / 3

# add a column for the average ram  
df['ram_avg'] = (df['ram_min'] + df['ram_max']*2) / 3

# add a column for the average storage
df['storage_avg'] = (df['storage_min'] + df['storage_max']*2) / 3

# make all columns string
df = df.astype(str)


# Define the non-existing phone specs
non_existing_phone_specs = {
    'details': '23000' ,
}

# Combine the pipelines only for those non-existing phone specs keys


feature_union_keys = []
for key in non_existing_phone_specs.keys():
    if key in df.columns:
        feature_union_keys.append(
            (key, Pipeline([
                ('selector', FunctionTransformer(lambda x: x[key], validate=False)),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ]))
        )


# Combine the pipelines using FeatureUnion
feature_union = FeatureUnion(feature_union_keys)

# Fit and transform the data
feature_vectors = feature_union.fit_transform(df)

# Now feature_vectors contains TF-IDF representations for both 'brand' and 'model'
from sklearn.neighbors import NearestNeighbors
# Create a DataFrame for the non-existing phone specs
non_existing_df = pd.DataFrame([non_existing_phone_specs])

# Transform the non-existing phone specs using the previously defined FeatureUnion
non_existing_feature_vectors = feature_union.transform(non_existing_df)

# Create a NearestNeighbors model
knn = NearestNeighbors(n_neighbors=20)

# Fit the feature vectors to the knn model
knn.fit(feature_vectors)

# Find the nearest neighbors for the non-existing phone specs
distances, indices = knn.kneighbors(non_existing_feature_vectors)

# Print top recommendation
print("Top Recommendation based on Non-Existing Phone Specs:")
df[['model', 'brand', 'release', 'colors', 'price_avg']].iloc[indices[0]]


Top Recommendation based on Non-Existing Phone Specs:


,model,brand,release,colors,price_avg
630,Poco X5,Xiaomi,2023,Jaguar Black|Wildcat Blue|Supernova Green,14165.666666666666
226,Moto G 5G Plus,Motorola,2020,Surfing Blue|Mystic Lilac,15109.666666666666
379,GT Master,Realme,2021,Black|Gray|White|Aurora,27013.0
174,Hot 20S,Infinix,2022,Sonic Black|Tempo Blue|Fantasy Purple|Light-ri...,21725.03667293065
428,Galaxy A33 5G,Samsung,2022,Black|White|Blue|Peach,17676.313666666665
328,C35,Oukitel,2023,Black|Green|Light Blue,12919.0
213,Edge 20 Fusion,Motorola,2021,Electric Graphite|Cyber Teal,16350.833333333334
103,Pixel 4a 5G,Google,2020,Just Black|Clearly White,16206.146999999999
66,KingKong 9,Cubot,2023,Black,19798.405
608,Mix 4,Xiaomi,2021,Ceramic Black|Ceramic White|Shadow Gray,32610.0


# THE COSHEN 1

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def get_top_recommendation(non_existing_phone_specs):
    # Sample data
    data = pd.read_csv('clean_phone_data.csv')
    df = pd.DataFrame(data)

    # add a column for the average price
    df['price_avg'] = (df['price_min'] + df['price_max']*2) / 3

    # add a column for the average ram  
    df['ram_avg'] = (df['ram_min'] + df['ram_max']*2) / 3

    # add a column for the average storage
    df['storage_avg'] = (df['storage_min'] + df['storage_max']*2) / 3


    df['price_range'] = pd.cut(df['price_avg'], bins=[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 100000000], labels=['<10k', '10k-20k', '20k-30k', '30k-40k', '40k-50k', '50k-60k', '60k-70k', '70k-80k', '80k-90k', '90k-100k', '>100k'])

    
    

    # make all columns string
    df = df.astype(str)

    # Define the non-existing phone specs
    non_existing_phone_specs = non_existing_phone_specs

    # Combine the pipelines only for those non-existing phone specs keys
    feature_union_keys = []
    for key in non_existing_phone_specs.keys():
        if key in df.columns:
            feature_union_keys.append(
                (key, Pipeline([
                    ('selector', FunctionTransformer(lambda x: x[key], validate=False)),
                    ('tfidf', TfidfVectorizer(stop_words='english'))
                ]))
            )

    # Combine the pipelines using FeatureUnion
    feature_union = FeatureUnion(feature_union_keys)

    # Fit and transform the data
    feature_vectors = feature_union.fit_transform(df)

    # Now feature_vectors contains TF-IDF representations for both 'brand' and 'model'
    from sklearn.neighbors import NearestNeighbors

    # Create a DataFrame for the non-existing phone specs
    non_existing_df = pd.DataFrame([non_existing_phone_specs])

    # Transform the non-existing phone specs using the previously defined FeatureUnion
    non_existing_feature_vectors = feature_union.transform(non_existing_df)

    # Create a NearestNeighbors model
    knn = NearestNeighbors(n_neighbors=10, metric='minkowski')

    # Fit the feature vectors to the knn model
    knn.fit(feature_vectors)

    # Find the nearest neighbors for the non-existing phone specs
    distances, indices = knn.kneighbors(non_existing_feature_vectors)

    # Print top recommendation
    top_recommendation = df.iloc[indices[0]]
    return top_recommendation

# Example usage:
non_existing_phone_specs = {
    'storage_avg': '128',
    'price_range': '<10k',
}

get_top_recommendation(non_existing_phone_specs)[:10]





,phone,brand,colors,model,battery_size,battery_endurance,weight,os,release,screen_size,...,lazada_reviews,build_material,has_wifi,has_bluetooth,has_cardslot,details,price_avg,ram_avg,storage_avg,price_range
60,coolpad_cool_20-11620,Coolpad,Black|Blue|White,Cool 20,4500,92.76704170684316,199.6,Android,2021,6.52,...,7,Glass|plastic|plastic,a|b|g|n|ac,5.0,Yes,//www.lazada.com.ph/products/2021-coolpad-cool...,9606.166666666666,5.333333333333333,106.66666666666667,<10k
34,blackview_a50-11588,Blackview,Black|Green|Blue|Gradient,A50,4280,88.2317641122864,156.0,Android,2022,6.01,...,0,Glass|plastic|plastic,b|g|n|ac,5.0,Yes,//www.lazada.com.ph/products/blackview-a50-3gb...,6653.0,3.0,64.0,<10k
37,blackview_a95-11572,Blackview,Black|Gold|Blue,A95,4380,90.29325392799404,195.0,Android,2022,6.53,...,8,Glass|plastic|plastic,b|g|n|ac,4.2,Yes,//www.lazada.com.ph/products/global-version-bl...,9098.0,8.0,128.0,<10k
59,coolpad_cool_20+-12421,Coolpad,Black|Blue,Cool 20+,4500,92.76704170684316,197.1,Android,2023,6.52,...,0,Glass|glass,a|b|g|n|ac,5.0,Yes,//www.lazada.com.ph/products/coolpad-cool-20-h...,7121.333333333333,5.333333333333333,128.0,<10k
128,honor_x5_plus-12645,Honor,Cyan Lake|Midnight Black,X5 Plus,5200,107.19747041679656,188.0,Android,2023,6.56,...,29,Glass|plastic|plastic,a|b|g|n|ac,5.1,Yes,//www.lazada.com.ph/products/honor-x5-plus-4gb...,5756.666666666667,4.0,64.0,<10k
98,doogee_x97_pro-11815,Doogee,Black|Blue|Green,X97 Pro,4200,86.5825722597203,205.747181372549,Android,2022,6.0,...,0,Glass|plastic|plastic,a|b|g|n|ac,5.0,Yes,//www.lazada.com.ph/products/doogee-x97-pro-4g...,9766.333333333334,4.0,64.0,<10k
102,google_pixel_4a-10123,Google,Just Black|Barely Blue,Pixel 4a,3140,76.0,143.0,Android,2020,5.81,...,22,Glass|Glass|plastic|plastic,a|b|g|n|ac,5.0,No,//www.lazada.com.ph/products/google-pixel-4a5g...,8282.235,6.0,128.0,<10k
130,honor_x6a-12442,Honor,Cyan Lake|Midnight Black,X6a,5200,107.19747041679656,188.0,Android,2023,6.56,...,13,Glass|plastic|plastic,a|b|g|n|ac,5.1,Yes,//www.lazada.com.ph/products/honor-x6a-4gb-128...,8271.666666666666,5.333333333333333,128.0,<10k
101,google_pixel_4-9896,Google,Clearly White|Just Black|Oh So Orange,Pixel 4,2800,62.0,162.0,Android,2019,5.7,...,53,Glass|Glass|glass|Glass|aluminum,a|b|g|n|ac,5.0,No,//www.lazada.com.ph/products/google-pixel-4-an...,8271.013666666666,6.0,106.66666666666667,<10k
55,blackview_oscal_c70-12267,Blackview,Dim Forest Green|Summer Sky Blue|Stardust Grey,Oscal C70,5180,106.78517245365504,192.0,Android,2023,6.56,...,0,Glass|plastic|plastic,a|b|g|n|ac,5.0,Yes,//www.lazada.com.ph/products/blackview-oscal-c...,8272.666666666666,6.0,128.0,<10k


In [162]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Sample phone data (replace with actual data)
phone_data = [
    {"brand": "Apple", "color": "Red|Black|White", "price": 12000, "model": "iPhone XR"},
    {"brand": "Apple", "color": "Blue", "price": 13000, "model": "iPhone 14 Plus"},
    {"brand": "Apple", "color": "Red", "price": 11500, "model": "iPhone 14"},
    {"brand": "Apple", "color": "Black", "price": 10000, "model": "iPhone 12"},
    {"brand": "Samsung", "color": "Red|Black|Violet", "price": 9000, "model": "Galaxy S20"},
    {"brand": "Samsung", "color": "Blue", "price": 9500, "model": "Galaxy S21"},
    {"brand": "Samsung", "color": "Black", "price": 8500, "model": "Galaxy S22"},
    {"brand": "Samsung", "color": "Red", "price": 8000, "model": "Galaxy S23"},
    {"brand": "Xiaomi", "color": "Red", "price": 7000, "model": "Redmi Note 10"},
    {"brand": "Xiaomi", "color": "Blue", "price": 7500, "model": "Redmi Note 11"},
    {"brand": "Xiaomi", "color": "Black", "price": 6500, "model": "Redmi Note 12"},
    {"brand": "Xiaomi", "color": "Red", "price": 6000, "model": "Redmi Note 13"},
    {"brand": "OnePlus", "color": "Red", "price": 11000, "model": "OnePlus 9"},
    {"brand": "OnePlus", "color": "Blue", "price": 11500, "model": "OnePlus 10"},
    {"brand": "OnePlus", "color": "Black", "price": 10500, "model": "OnePlus 11"},
    {"brand": "OnePlus", "color": "Red", "price": 10000, "model": "OnePlus 12"},
    {"brand": "Google", "color": "Red", "price": 8000, "model": "Pixel 4"},
    {"brand": "Google", "color": "Blue", "price": 8500, "model": "Pixel 5"},
    {"brand": "Google", "color": "Black", "price": 7500, "model": "Pixel 6"},
    {"brand": "Google", "color": "Red", "price": 7000, "model": "Pixel 7"},
    {"brand": "Sony", "color": "Red", "price": 9000, "model": "Xperia 1"},
    {"brand": "Sony", "color": "Blue", "price": 9500, "model": "Xperia 2"},
    {"brand": "Sony", "color": "Black", "price": 8500, "model": "Xperia 3"},
    {"brand": "Sony", "color": "Red", "price": 8000, "model": "Xperia 4"},
    {"brand": "Oppo", "color": "Red", "price": 7000, "model": "A53"},
    {"brand": "Oppo", "color": "Blue", "price": 7500, "model": "A54"},
    {"brand": "Oppo", "color": "Black", "price": 6500, "model": "A55"},
    {"brand": "Oppo", "color": "Red", "price": 6000, "model": "A56"}
]

# Encode categorical features
brand_encoding = {"Apple": 1, "Samsung": 2, "Xiaomi": 3, "OnePlus": 4, "Google": 5, "Sony": 6, "Oppo": 7}  # Dummy encoding for demonstration
color_encoding = {"Red": 1, "Blue": 2, "Black": 3, "White": 4}  # Dummy encoding for demonstration

# Encode user preferences
user_preferences = {"brand": "Samsung", "color": "Violet", "price": 9000}

# Function to calculate cosine similarity between user preferences and phone features
def calculate_similarity(phone_features, user_preferences):
    phone_colors = phone_features["color"].split("|")
    phone_vector = np.array([
        brand_encoding.get(phone_features["brand"], 0),
        max([color_encoding.get(color, 0) for color in phone_colors]),
        phone_features["price"]
    ]).reshape(1, -1)
    user_vector = np.array([
        brand_encoding.get(user_preferences["brand"], 0),
        color_encoding.get(user_preferences["color"], 0),
        user_preferences["price"]
    ]).reshape(1, -1)
    return cosine_similarity(phone_vector, user_vector)[0][0]

# Calculate similarity scores for all phones
similarity_scores = []
for phone in phone_data:
    similarity_scores.append((phone["model"], calculate_similarity(phone, user_preferences)))

# Sort phones by similarity score
recommended_phones = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Print recommended phones
print("----- Recommended Phones -----")
for phone, score in recommended_phones:
    print(f"{phone} (Similarity Score: {score})")


----- Recommended Phones -----
Galaxy S23 (Similarity Score: 0.9999999918016983)
iPhone 14 (Similarity Score: 0.9999999870708774)
OnePlus 9 (Similarity Score: 0.9999999858687914)
OnePlus 12 (Similarity Score: 0.999999979197535)
Galaxy S21 (Similarity Score: 0.9999999777709399)
iPhone 14 Plus (Similarity Score: 0.9999999776097607)
OnePlus 10 (Similarity Score: 0.9999999769889648)
Redmi Note 10 (Similarity Score: 0.999999968505928)
Redmi Note 11 (Similarity Score: 0.9999999486419867)
Redmi Note 13 (Similarity Score: 0.9999999475308793)
iPhone 12 (Similarity Score: 0.9999999475308684)
OnePlus 11 (Similarity Score: 0.9999999465860531)
Galaxy S20 (Similarity Score: 0.9999999444444518)
Galaxy S22 (Similarity Score: 0.9999999376308351)
iPhone XR (Similarity Score: 0.9999999347993888)
Pixel 4 (Similarity Score: 0.9999999110725669)
Pixel 5 (Similarity Score: 0.9999999053355917)
Xperia 1 (Similarity Score: 0.9999998950617759)
Xperia 2 (Similarity Score: 0.9999998940529163)
Pixel 7 (Similarity Sc